### Wine regions in Poland

This project aims to find wine regions in Poland and describe meteorological conditions observed over them.\
It is going to cover topics like:
- Find wine regions in Poland
- Analyze meteorological conditions over discovered regions
- Present how to use spatial-econometrics method to find spatial dependencies between geographical objects 
- Present how to obtain and analyze open meteorological data
- Have fun with data ;)

In [ ]:
import geopandas as gpd
import libpysal as ps
from esda.moran import Moran_Local
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import FancyArrowPatch
from sklearn.cluster import KMeans
from matplotlib.lines import Line2D
from shapely.geometry import MultiPolygon

In [ ]:
gminy = gpd.read_file('shp/gminy.shp')

In [ ]:
w_weights = ps.weights.Queen.from_dataframe(gminy)
moran_loc = Moran_Local(gminy['winnice'], w_weights)

# Local Moran's I value
local_moran_i = moran_loc.Is
gminy['Local_Moran_I'] = local_moran_i

In [ ]:
# Create a function to classify values into four categories
def classify_moran_i(value, threshold):
    if value >= threshold:
        return 'HH'
    else:
        return 'LL'

# Define a threshold for HIGH values (2 is based on general knowledge on the topic)
hh_threshold = 2

gminy['Moran_Class'] = [classify_moran_i(i, hh_threshold) for i in local_moran_i]

# Update classification for LH and HL based on Moran's I values
for i in range(len(gminy)):
    if local_moran_i[i] < 0 and gminy['Moran_Class'][i] == 'HH':
        gminy['Moran_Class'][i] = 'LH'
    elif local_moran_i[i] < 0 and gminy['Moran_Class'][i] == 'LL':
        gminy['Moran_Class'][i] = 'HL'

In [ ]:
# Create a map of the classification
fig, ax = plt.subplots(figsize=(8, 8))
gminy.plot(column='Moran_Class', cmap='coolwarm', ax=ax, legend=True)
ax.set_title('Local Moran\'s I Classification (Threshold = {})'.format(hh_threshold))

plt.show()